In [1]:
import pandas as pd
import osmnx as ox
import networkx as nx
import math
from numpy import linalg as LA
import numpy as np

C:\Users\Willemijn Tutuarima\miniconda3\envs\gds\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


# File to calculate the weights of edges

In [2]:
filepath="../graph/rotterdam_drive_bbox_cameras_traffic_lights_bridges_roundabouts_tunnels.graphml"

In [3]:
graph=ox.load_graphml(filepath)

## Start with default

In [6]:
for road_id, (origin_num, destination_num, data) in enumerate(graph.edges(data=True)):
    if isinstance(data.get('maxspeed'),list):
        nx.set_edge_attributes(graph,
                               {(origin_num, destination_num, 0): {
                                   "default": data.get('length') / float(data.get('maxspeed')[0])},
                                (origin_num, destination_num, 1): {
                                    "default": data.get('length') / float(data.get('maxspeed')[1])}})

    elif isinstance(data.get('maxspeed'),str):
        nx.set_edge_attributes(graph,
                           {(origin_num, destination_num, 0): {"time_to_cross": data.get('length') / float(data.get('maxspeed'))},
                            (origin_num, destination_num, 1): {"time_to_cross": data.get('length') / float(data.get('maxspeed'))}})

    else:
        # if maximum speed is not specified, max speed of 30 km/h is assumed
        # the number of edges without maximum speed is 2402, from the total of 25348 edges (so 9.47%)
        nx.set_edge_attributes(graph,
                               {(origin_num, destination_num, 0): {
                                   "default": data.get('length') / 30.0},
                                (origin_num, destination_num, 1): {
                                    "default": data.get('length') / 30.0}})

## Camera avoidance 

In [8]:
for road_id, (origin_num, destination_num, data) in enumerate(graph.edges(data=True)):
    if isinstance(data.get("camera"), bool):
        #Low
        nx.set_edge_attributes(graph,
                               {(origin_num, destination_num, 0): {
                                   "CA1OA0LP0RP0WW0HS0SR0TA0": data.get('default') * 10 },
                                (origin_num, destination_num, 1): {
                                    "CA1OA0LP0RP0WW0HS0SR0TA0": data.get('default') * 10}})
        
        #Medium
        nx.set_edge_attributes(graph,
                               {(origin_num, destination_num, 0): {
                                   "CA2OA0LP0RP0WW0HS0SR0TA0": data.get('default') * 50},
                                (origin_num, destination_num, 1): {
                                    "CA2OA0LP0RP0WW0HS0SR0TA0": data.get('default') * 50}})
        
        #High
        
        nx.set_edge_attributes(graph,
                               {(origin_num, destination_num, 0): {
                                   "CA3OA0LP0RP0WW0HS0SR0TA0": data.get('default') * 100},
                                (origin_num, destination_num, 1): {
                                    "CA3OA0LP0RP0WW0HS0SR0TA0": data.get('default') * 100}})

## obstacle avoidance

In [ ]:
for road_id, (origin_num, destination_num, data) in enumerate(graph.edges(data=True)):
    if isinstance(data.get("tunnel"), bool) | isinstance(data.get("roundabout"), bool) | isinstance(data.get("bridge"), bool) | isinstance(data.get("traffic_light"), bool):
        #Low
        nx.set_edge_attributes(graph,
                               {(origin_num, destination_num, 0): {
                                   "CA0OA1LP0RP0WW0HS0SR0TA0": data.get('default') * 1.2 },
                                (origin_num, destination_num, 1): {
                                    "CA0OA1LP0RP0WW0HS0SR0TA0": data.get('default') * 1.2}})
        
        #Medium
        nx.set_edge_attributes(graph,
                               {(origin_num, destination_num, 0): {
                                   "CA0OA2LP0RP0WW0HS0SR0TA0": data.get('default') * 1.4},
                                (origin_num, destination_num, 1): {
                                    "CA0OA2LP0RP0WW0HS0SR0TA0": data.get('default') * 1.4}})
        
        #High
        
        nx.set_edge_attributes(graph,
                               {(origin_num, destination_num, 0): {
                                   "CA0OA3LP0RP0WW0HS0SR0TA0": data.get('default') * 1.6},
                                (origin_num, destination_num, 1): {
                                    "CA0OA3LP0RP0WW0HS0SR0TA0": data.get('default') * 1.6}})

## Lane preference

In [ ]:
for road_id, (origin_num, destination_num, data) in enumerate(graph.edges(data=True)):
    if 'lanes' in data:
        if int(data.get('lanes')) > 1:
            #Low
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP1RP0WW0HS0SR0TA0": data.get('default') * 0.9 },
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP1RP0WW0HS0SR0TA0": data.get('default') * 0.9}})

            #Medium
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP2RP0WW0HS0SR0TA0": data.get('default') * 0.8},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP2RP0WW0HS0SR0TA0": data.get('default') * 0.8}})

            #High

            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP3RP0WW0HS0SR0TA0": data.get('default') * 0.7},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP3RP0WW0HS0SR0TA0": data.get('default') * 0.7}})

## Residential preference

In [ ]:
for road_id, (origin_num, destination_num, data) in enumerate(graph.edges(data=True)):
    if 'highway' in data:
        if data.get('highway') == "residential":
            #Low pos
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP1WW0HS0SR0TA0": data.get('default') * 0.9 },
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP1WW0HS0SR0TA0": data.get('default') * 0.9}})

            #Medium pos
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP2WW0HS0SR0TA0": data.get('default') * 0.8},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP2WW0HS0SR0TA0": data.get('default') * 0.8}})

            #High pos

            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP3WW0HS0SR0TA0": data.get('default') * 0.7},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP3WW0HS0SR0TA0": data.get('default') * 0.7}})
            
            #low neg
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP4WW0HS0SR0TA0": data.get('default') * 1.2},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP4WW0HS0SR0TA0": data.get('default') * 1.2}})

            #Medium neg
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP4WW0HS0SR0TA0": data.get('default') * 1.4},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP4WW0HS0SR0TA0": data.get('default') * 1.4}})

            #High neg

            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP4WW0HS0SR0TA0": data.get('default') * 1.6},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP4WW0HS0SR0TA0": data.get('default') * 1.6}})

## Wrong way 

In [ ]:
for road_id, (origin_num, destination_num, data) in enumerate(graph.edges(data=True)):
    if 'oneway' in data:
        if data.get('oneway'):
            #Low pos
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP0WW1HS0SR0TA0": data.get('default') * 0.9},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP0WW1HS0SR0TA0": data.get('default') * 0.9}})

            #Medium pos 
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP0WW2HS0SR0TA0": data.get('default') * 0.8},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP0WW2HS0SR0TA0": data.get('default') * 0.8}})

            #High pos

            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP0WW3HS0SR0TA0": data.get('default') * 0.7},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP0WW3HS0SR0TA0": data.get('default') * 0.7}})
            
            #low neg
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP0WW4HS0SR0TA0": data.get('default') * 1.2},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP0WW4HS0SR0TA0": data.get('default') * 1.2}})

            #Medium neg
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP0WW5HS0SR0TA0": data.get('default') * 1.4},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP0WW5HS0SR0TA0": data.get('default') * 1.4}})

            #High neg
            nx.set_edge_attributes(graph,
                                   {(origin_num, destination_num, 0): {
                                       "CA0OA0LP0RP0WW6HS0SR0TA0": data.get('default') * 1.6},
                                    (origin_num, destination_num, 1): {
                                        "CA0OA0LP0RP0WW6HS0SR0TA0": data.get('default') * 1.6}})